In [187]:
import pandas as pd
import datetime
import time
from dateutil.parser import parse
import json

FALL_YEAR = 2017
SPRING_YEAR = 2018
SUMMER_YEAR = 2018

In [188]:
html = pd.read_html('http://www.upenn.edu/almanac/3yearcal.html', parse_dates=True)

In [189]:
relevant_rows = pd.DataFrame(html[0].iloc[29:74, 0:5])

#separate the rows by term
fall = relevant_rows.loc[30:49]
spring = relevant_rows.loc[51:65]
summer = relevant_rows.loc[67:73]

# Only grab values for Fall 2017
fall.drop([1,3,4], axis = 1, inplace = True)
spring.drop([1,3,4], axis = 1, inplace = True)
summer.drop([1,3,4], axis = 1, inplace = True)

#remove integer indexing
fall.set_index(0,inplace=True)
spring.set_index(0,inplace=True)
summer.set_index(0,inplace=True)

#rename columns
fall.rename(index=str, columns = {2:'dates'}, inplace=True)
spring.rename(index=str, columns = {2:'dates'}, inplace=True)
summer.rename(index=str, columns = {2:'dates'}, inplace=True)


/Users/niharpatil/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/niharpatil/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/niharpatil/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by Interac

In [190]:
#returns [start_date, end_date] for every event
def split_date_by_hyphen(datestring):
    date_range = datestring.split('-')
    if(len(date_range) == 1):
        return [datestring, datestring]
    start_date = date_range[0]
    end = date_range[1]
    try:
        end_number = int(end)
        month = start_date.split(' ')[0]
        end_date = month + ' ' + str(end_number)
        return [start_date, end_date]
    except ValueError:
        return [start_date, end]

In [191]:
#stuff cleaning 2017-2018 rows
fall.loc['Homecoming'] = 'November 4'
spring.iloc[0] = 'January 10'

/Users/niharpatil/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/niharpatil/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [192]:
def extract_events(df): 
    df = df.copy(deep=True)
    events = []
    for index, row in df.iterrows():
        date = row['dates']
        date_range = split_date_by_hyphen(date)
        start_date_str = str(parse(date_range[0]))
        end_date_str = str(parse(date_range[1]))

        events.append({
            "event": index,
            "start": start_date_str,
            "end": end_date_str
        })

    return events
    
events = extract_events(fall) + extract_events(spring) + extract_events(summer)

with open('events.json', 'wb') as outfile:
    json.dump(events, outfile)